In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

# Instantiating the Webdriver
I used Selenium + chromedriver to open an automated webdriver for chrome. To get information from the pages, I used a combination of BeautifulSoup and Selenium driver functions.

In [517]:
agents = pd.read_csv('agents.csv')
driver.quit()

In [518]:
driver.quit()
useragent = agents['0'][58]
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={useragent}")
options.add_argument('--incognito')
chromedriver = '~/Downloads/chromedriver'
chromedriver = os.path.expanduser(chromedriver)
driver = webdriver.Chrome(chromedriver, chrome_options=options)
driver.delete_all_cookies()

C:\Users\JChaotogo\Anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


# Functions to collect URLs
Collects URLs for the listings that I wanted to analyze. Also contains functions that I needed in order to make various clicks as well as notify me of a captcha.

In [403]:
def get_house_links(url, driver, pages=20):
    house_links=[]
    driver.get(url)
    for i in range(pages):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(np.random.lognormal(0,1))
        listings = soup.find_all("a", class_="zsg-photo-card-overlay-link")
        page_data = ['https://www.zillow.com'+row['href'] for row in listings]
        house_links.append(page_data)
        next_button = soup.find_all("a", class_="on")
        next_button_link = ['https://www.zillow.com'+row['href'] for row in next_button]
        if i<19:
            time.sleep(2)
            WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME , 'zsg-pagination-next'))).click()


    return house_links

def get_html_data(url, driver):
    driver.get(url)
    time.sleep(np.random.lognormal(0,1))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def flatten_list(house_links):
    house_links_flat=[]
    for sublist in house_links:
        for item in sublist:
            house_links_flat.append(item)
    return house_links_flat

def _is_element_displayed(driver, elem_text, elem_type):
    if elem_type == "class":
        try:
            out = driver.find_element_by_class_name(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    elif elem_type == "css":
        try:
            out = driver.find_element_by_css_selector(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    else:
        raise ValueError("arg 'elem_type' must be either 'class' or 'css'")
    return(out)


def _pause_for_captcha(driver):
    while True:
        time.sleep(10)
        if not _is_element_displayed(driver, "captcha-container", "class"):
            break

# Check to see if the page is currently stuck on a captcha page. If so, pause 
# the scraper until user has manually completed the captcha requirements.
def check_for_captcha(driver):
    if _is_element_displayed(driver, "captcha-container", "class"):
        print("\nCAPTCHA!\n"\
              "Manually complete the captcha requirements.\n"\
              "Once that's done, if the program was in the middle of scraping "\
              "(and is still running), it should resume scraping after ~30 seconds.")
        _pause_for_captcha(driver)

# Functions for specific information

There were 2 formats for some listing pages. I wrote functions for both and used try and except to choose the correct functions to run.

In [202]:
def get_tax(driver):
    try:
        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.ID , 'price-and-tax-history'))).click()
        tabs = WebDriverWait(driver,5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".zsg-tab-link")))
        tabs[1].click()
        text = driver.find_element_by_css_selector('#hdp-tax-history table').text
        tax = text.split('\n')[1]
        return tax
    except:
        return np.nan

def get_percent_increase(driver):
    try:
        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.ID , 'homeValue'))).click()
        text = driver.find_element_by_class_name('secondary-zestimate-items').text
        forecast = text.split('\n')[-1]
        return forecast
    except:
        return np.nan

def get_hoa_fee(soup):
    try:
        hoa = re.findall('HOA Fee","factValue":"\$[0-9]{2,3}', soup.text)
        return hoa
    except:
        return np.nan

def get_numbaths(soup):
    try:
        baths = soup.find('h3', class_='edit-facts-light')
        baths = baths.find_all('span')[3].text
        baths = baths.replace('baths', '').lower().strip()
        return float(baths)
    except:
        return np.nan
    
def get_numbeds(soup):
    try:
        beds = soup.find('h3', class_='edit-facts-light')
        beds = beds.find_all('span')[1].text
        beds = beds.replace('beds', '').lower().strip()
        return float(beds)
    except:
        return np.nan

def get_floor_size(soup):
    try:
        sqft = soup.find('h3', class_='edit-facts-light')
        sqft = sqft.find_all('span')[5].text
        sqft = sqft.replace('sqft', '').replace(',', '').lower().strip()
        return sqft
    except:
        return np.nan

def get_lot_size(soup):
    try:
        price = soup.find_all('div', class_='zestimate-value')[5].text
        price = price.replace(',', '').replace('sqft', '').lower().strip()
        return int(price)
    except:
        return np.nan

def get_property_type(soup):
    try:
        price = soup.find_all('div', class_='fact-value')[0].text
        price = price.lower().strip()
        return str(price)
    except:
        return np.nan

def get_year_built(soup):
    try:
        price = soup.find_all('div', class_='fact-value')[1].text
        price = price.lower().strip()
        return int(price)
    except:
        return np.nan

def get_zestimate(soup):
    try:
        price = soup.find('div', class_='zestimate-value').text
        return price
    except:
        return np.nan
    
def get_rent(soup):
    try:
        price = soup.find_all('div', class_='toggle-section')[-1]
        price = price.find('div', class_='zestimate-value').text
        return price
    except:
        return np.nan
    
def get_sale_price(soup):
    try:
        price = soup.find('span', class_='value').text
        price = price.replace('$', '').replace(',', '').lower().strip()
        return int(price)
    except:
        return np.nan

    
def get_zip(soup):
    try:
        zipcode = soup.find('div', class_='zsg-h2').text
        zipcode = re.search('[0-9]{5}', zipcode)
        zipcode = zipcode.group(0)
        return int(zipcode)
    except:
        return 'None'
    
def get_address(soup):
    try:
        address = soup.find('div', class_='zsg-h1 hdp-home-header-st-addr').text.strip().lower()
        return address
    except:
        return 'None'

    
def get_city(soup):
    try:
        city = soup.find('div', class_='zsg-h2').text.strip().lower()
        city = city.split(',')[0]
        return str(city)
    except:
        return 'None'
    

In [451]:

def get_zestimate1(soup):
    try:
        zest = soup.find_all('h4', class_='zestimate-value')
        zestimate = zest[0].text
        rent = zest[1].text
        return zestimate, rent
    except:
        return np.nan, np.nan
def get_bed_bath_live1(soup):
    try:
        info = soup.find('h3', class_='ds-bed-bath-living-area-container')
        info = info.find_all('span')
        beds = info[0].text
        baths= info[3].text
        sqft = info[6].text
        return beds, baths, sqft
    except:
        return np.nan, np.nan, np,nan
def get_tax1(soup):
    try:
        tax = soup.find('tr', class_='ds-tax-table-row').text
        return tax
    except:
        return np.nan
    
def get_hoa1(soup):
    try:
        hoa = soup.find_all('div', class_='sc-jKJlTe loXQau')[4].text
        return hoa
    except:
        return np.nan
def get_percent_increase1(soup):
    try:
        percent = soup.find_all('span', class_='percent-increase')[-1].text
        return percent
    except:
        return np.nan
def get_sale_price1(soup):
    try:
        price = soup.find('span', class_='ds-value').text
        return price
    except:
        return np.nan
def get_property_type1(soup):
    try:
        prop = soup.find('span', class_='ds-body ds-home-fact-value').text
        return prop
    except:
        return np.nan
def get_year_built1(soup):
    try:
        year = soup.find_all('span', class_='ds-body ds-home-fact-value')[1].text
        return year
    except:
        return np.nan
def get_address1(soup):
    try:
        full = soup.find('h1', class_='ds-address-container')
        address = full.find_all('span')[0].text
        city = full.find_all('span')[1].text.split(',')[0]
        zipcode = full.find_all('span')[1].text
        zipcode = re.findall('[0-9]{5}', zipcode)
        return address, city, zipcode
    except:
        return np.nan, np.nan, np.nan
    
def get_house_data1(driver, house_links_flat, houselist):
    for link in tqdm(house_links_flat):
        soup = get_html_data(link, driver)
        check_for_captcha(driver)
        try:
            address, city, zipcode = get_address1(soup)
            beds, baths, sqft = get_bed_bath_live1(soup)
            year = get_year_built1(soup)
            saleprice = get_sale_price1(soup)
            zestimate, rent = get_zestimate1(soup)
            property_type = get_property_type1(soup)
            tax = get_tax1(soup)
            hoa = get_hoa1(soup)
            percent_increase = get_percent_increase1(soup)
            houselist.append([address, city, zipcode, beds, baths, sqft, year,
                             saleprice, zestimate, rent, property_type, hoa,
                             tax, percent_increase, link])
            time.sleep(np.random.lognormal(0,1)+2)
        except:
            city = get_city(soup)
            zipcode = get_zip(soup)
            beds = get_numbeds(soup)
            baths = get_numbaths(soup)
            floor_size = get_floor_size(soup)
            year_built = get_year_built(soup)
            saleprice = get_sale_price(soup)
            rent = re.findall('"rentZestimate":[0-9]{3,4}', soup.text)
            zestimate = re.findall('"zestimate":[0-9]{5,8}', soup.text)
            property_type = get_property_type(soup)
            tax = get_tax(driver)
            hoa = get_hoa_fee(soup)
            check_for_captcha(driver)
            percent_increase = get_percent_increase(driver)
            houselist.append([address, city, zipcode, beds, baths, floor_size, year_built, saleprice, zestimate,
                              rent, property_type, hoa, tax, percent_increase, link])
            time.sleep(np.random.lognormal(0,1)+2)
            

In [472]:
fortworthurl = 'https://www.zillow.com/homes/for_sale/Austin-TX/house,condo,apartment_duplex,townhouse_type/10221_rid/2-_beds/100000-325000_price/393-1279_mp/1960-_built/0-100_hoa/globalrelevanceex_sort/30.538016,-97.416459,30.048887,-98.154603_rect/10_zm/'
fort20 = get_house_links(fortworthurl, driver, pages=20)
austinlinks = flatten_list(fort20)

In [420]:
atlantahomes=[]

In [469]:
get_house_data1(driver, atlantalinks[486:], atlantahomes)












































  0%|          | 0/14 [00:00<?, ?it/s]










































  7%|▋         | 1/14 [00:17<03:46, 17.42s/it]










































 14%|█▍        | 2/14 [00:32<03:21, 16.83s/it]










































 21%|██▏       | 3/14 [00:49<03:03, 16.66s/it]










































 29%|██▊       | 4/14 [01:02<02:36, 15.68s/it]










































 36%|███▌      | 5/14 [01:17<02:18, 15.34s/it]










































 43%|████▎     | 6/14 [01:28<01:53, 14.19s/it]










































 50%|█████     | 7/14 [01:38<01:30, 12.90s/it]










































 57%|█████▋    | 8/14 [01:49<01:14, 12.46s/it]










































 64%|██████▍   | 9/14 [02:02<01:02, 12.42s/it]










































 71%|███████▏  | 10/14 [02:13<00:48, 12.18s/it]



























